In [ ]:
%%capture
!pip install -U fashion-clip

In [ ]:
import sys
# sys.path.append("fashion-clip/")
from fashion_clip.fashion_clip import FashionCLIP
import pandas as pd
import numpy as np
from collections import Counter
from PIL import Image
import requests
from io import BytesIO

In [ ]:
%%capture
fclip = FashionCLIP('fashion-clip')

In [ ]:
df = pd.read_pickle('preprocessed_data.pkl')

In [ ]:
df.shape

(5918, 5)

In [ ]:
import pickle
import numpy as np
from fashion_clip.fashion_clip import FashionCLIP
import tensorflow as tf
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import requests
import concurrent.futures
from itertools import zip_longest

def chunked(iterable, n):
    args = [iter(iterable)] * n
    return zip_longest(*args)

def process_image(args):
    image_url, name, url = args
    image_url = image_url[0]  # Assuming the image URL is the first element in the list

    # Load the image from the URL
    image = Image.open(requests.get(image_url, stream=True).raw)

    # Convert the image to RGB if it has a single channel
    if image.mode != "RGB":
        image = image.convert("RGB")

    return image, name, url

def classify_product(description):
    # Load the FashionCLIP model
    with open('fclip_model.pkl', 'rb') as file:
        loaded_model_data = pickle.load(file)

    loaded_image_embeddings = loaded_model_data['image_embeddings']
    loaded_text_embeddings = loaded_model_data['text_embeddings']
    loaded_images = loaded_model_data['images']

    fclip = FashionCLIP('fashion-clip')

    # Encode the query description
    # Encode multiple descriptions in a batch
    query_text_embedding = fclip.encode_text([description], batch_size=32)[0]

    # Calculate similarity scores between the query and loaded embeddings
    similarity_scores = np.dot(query_text_embedding, loaded_image_embeddings.T)

    # Sort indices based on similarity scores in descending order
    sorted_indices = np.argsort(similarity_scores)[::-1]

    # Get the URLs of the top K most similar images
    top_k = 15
    top_similar_image_urls = [loaded_images[i] for i in sorted_indices[:top_k]]

    # Create a new DataFrame containing information of products in top_similar_image_urls
    df_top_similar = df[df['images'].apply(lambda x: any(link in x for link in top_similar_image_urls))]

    # Get the URL, name, and image information of products in top_similar_image_urls
    result_urls = df_top_similar['url'].tolist()
    result_names = df_top_similar['name'].tolist()
    result_images = df_top_similar['images'].tolist()

    # Load the ViT model
    processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
    model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

    # List of class labels
    labels = model.config.id2label

    # Keyword lists for classification
    shirt_labels = ['belts', 'blazers', 'dresses', 'dupatta', 'jackets', 'kurtas', 'kurtis', 'lehenga choli', 'nehru jackets', 'rain jacket', 'bulletproof vest', 'cardigan',
           'rompers', 'shirts', 'shrug', 'suspenders', 'sweaters', 'sweatshirt', 'tops', 'T-shirt,', 'tunics', 'waistcoat', 'hoodie', 'jerrsey', 'tee shirt', 'maillot', 'fur coat'
           'wool', 'woolen', 'woollen', 'fur coat', 'brassiere', 'bra', 'bandeau', 'tank suit', 'brassiere', 'bandeau', 'sarong', 'pajama', 'pyjama', 'pj\'s', 'jammies', 'velvet',
           'bulletproof vest', 'maillot', 'tank suit', 'miniskirt', 'mini', 'sleeping bag', 'trench coat', 'lab coat', 'laboratory coat', 'velvet', 'fur coat', 'horizontal bar, high bar']
    pants_labels = ['capris', 'churidar', 'jeans', 'jeggings', 'leggings', 'patiala', 'salwar', 'salwar dupatta', 'shorts', 'skirts', 'stockings',
           'swimwear', 'tights', 'track pants', 'tracksuits', 'trousers', 'blue jean', 'jean', 'denim', 'swimming trunks', 'bathing trunks']
    shoes_labels = ['casual shoes', 'flats', 'flip flops', 'formal shoes', 'heels', 'sandal', 'holster', 'clog', 'geta', 'patten', 'sabot',
           'sandals', 'sports sandals', 'sports shoes', 'boot', 'running shoe', 'hair slide', 'sabot', 'cowboy hat', 'ten-gallon hat']

    # Predict and classify each image

    # Use concurrent processing to download images concurrently
    with concurrent.futures.ThreadPoolExecutor() as executor:
        image_results = list(executor.map(process_image, zip(result_images, result_names, result_urls)))

    class_dict = {}

    for (image, name, url), (image_url, _, _) in zip(image_results, zip(result_images, result_names, result_urls)):
        # Prepare the input data
        inputs = processor(images=np.array(image), return_tensors="pt")

        # Predict the class label
        outputs = model(**inputs)
        predicted_class_index = outputs.logits.argmax(dim=1).item()
        predicted_label = labels[predicted_class_index]

        # Classify the label as "pants", "shirt", "shoes", or "others"
        if any(label in predicted_label for label in pants_labels):
            predicted_label = "pants"
        elif any(label in predicted_label for label in shirt_labels):
            predicted_label = "shirt"
        elif any(label in predicted_label for label in shoes_labels):
            predicted_label = "shoes"
        else:
            predicted_label = "others"

        # Store the product information in the dictionary based on the label
        if predicted_label not in class_dict:
            class_dict[predicted_label] = []
        class_dict[predicted_label].append((name, image_url, url))

    return class_dict

In [ ]:
# Example usage
description = "summer outfit"
results = classify_product(description)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

1it [00:00,  6.00it/s]


In [ ]:
for label, products in results.items():
    print(f'Label: {label}')
    for name, image_url, url in products:
        print(f'Name: {name}')
        print(f'Image: {image_url}')
        print(f'URL: {url}')
        print("---")

Label: shirt
Name: Camo Shorts and Tee Set
Image: ['https://assets.adidas.com/images/w_600,f_auto,q_auto/9a8aac00270b43f08612e5cfdb20dfa8_9366/Camo_Shorts_and_Tee_Set_Black_IP0257_01_laydown.jpg']
URL: https://www.adidas.com/us/camo-shorts-and-tee-set/IP0257.html
---
Name: Nike Sportswear Collection
Image: ['https://static.nike.com/a/images/t_default/3817406d-9c4b-4c65-9881-75d1dd1359f7/sportswear-collection-womens-mock-neck-cropped-tank-NHbLf5.png']
URL: https://www.nike.com/t/sportswear-collection-womens-mock-neck-cropped-tank-NHbLf5/FB8343-309
---
Name: Nike
Image: ['https://static.nike.com/a/images/t_default/bf2540b9-8f71-4679-bef8-ffdb0ad3827d/baby-12-24m-tank-and-shorts-set-P9p54q.png']
URL: https://www.nike.com/t/baby-12-24m-tank-and-shorts-set-P9p54q/16J438-GB7
---
Name: Nike Coral Reef Tee and Shorts Set
Image: ['https://static.nike.com/a/images/t_default/c4c18b50-ffb6-45bc-95c7-396c02ff8efe/coral-reef-tee-and-shorts-set-baby-2-piece-dri-fit-set-QCBh8N.png']
URL: https://www.n

In [ ]:
import pickle

# Save the classify_product function as a pickle file
with open('classify_product.pkl', 'wb') as file:
    pickle.dump(classify_product, file)